In [24]:
import moleculekit.molecule as ht
import moleculekit.tools.voxeldescriptors as vd
import moleculekit.smallmol.smallmol as sm
from tqdm import *
from oddt import datasets
from moleculekit.tools.atomtyper import prepareProteinForAtomtyping
from moleculekit.tools.preparation import systemPrepare
from sklearn.model_selection import train_test_split
import numpy as np
import os
import h5py

In [25]:
# 设置路径
data_dir = ".\\data"
pdbbind_dir = os.path.join(data_dir, "refined-set-2016\\")
pdbbind_dataset = datasets.pdbbind(home=pdbbind_dir, default_set='refined', version=2016)

In [26]:
def get_pdb_complex_feature(pro_file, lig_file):
    """ 返回组合了protein和ligand的element feature体素化结果 """

    # 分别读取蛋白质分子的 .pdbqt 和小分子的 .mol2 文件
    pro_mol = ht.Molecule(pro_file)
    lig_mol = sm.SmallMol(lig_file)

    # 定义盒子大小和体素化中心（小分子的欧几里得中心）
    n = [24, 24, 24]
    center = list(sm.SmallMol.getCenter(lig_mol))

    # 体素化
    # 蛋白质准备
    _ = pro_mol.remove("element H")
    _ = pro_mol.filter('protein')
    try:
        pro_mol = prepareProteinForAtomtyping(pro_mol, verbose=False)
    except Exception as e:
        if 'ignore_ns_errors=True' in str(e):
            pro_mol = systemPrepare(pro_mol, ignore_ns_errors=True, verbose=False)
            pro_mol = prepareProteinForAtomtyping(pro_mol,protonate=False, verbose=False)
    
    # 蛋白质体素化过程
    pro_vox, pro_centers, pro_N = vd.getVoxelDescriptors(pro_mol, boxsize=n, center=center)
    p_features = pro_vox.reshape(pro_N[0], pro_N[1], pro_N[2], pro_vox.shape[1])
    # 小分子体素化过程
    lig_vox, lig_centers, lig_N = vd.getVoxelDescriptors(lig_mol, boxsize=n, center=center)
    l_features = lig_vox.reshape(lig_N[0], lig_N[1], lig_N[2], lig_vox.shape[1])

    # 组合两个体素化过后的数组
    features = np.concatenate((p_features, l_features), axis=3)
    
    return features

In [28]:
def get_pdb_features(ids, sets="refined"):
    """ 根据PDBid 进行体素化 并返回体素化结果x 和标签y """
    pdb_ids = []
    pdb_features = []

    # 逐一进行体素化，并使用tqdm生成进度条。
    for pdbid in tqdm(ids):
        # 根据PDBid获得文件路径
        protein_file = os.path.join(pdbbind_dir, pdbid, pdbid + "_protein.pdbqt")
        ligand_file = os.path.join(pdbbind_dir, pdbid, pdbid + "_ligand.mol2")

        # 若文件不存在则跳过
        if not os.path.isfile(protein_file) or not os.path.isfile(ligand_file):
            with open('.\\miss.log', 'a') as f:
                f.write(protein_file+ ' or '+ ligand_file+ ' not find\n')
            continue
        # 若文件不能体素化则输出错误，并跳过
        try:
            features = get_pdb_complex_feature(protein_file, ligand_file)
        except Exception as e:
            print("ERROR in ", pdbid , " ", str(e))
            with open('.\\error.log', 'a') as f:
                f.write("ERROR in "+ pdbid+ " "+ str(e)+ '\n')
            continue
        
        # 对应获得PDBid的列表和element feature的数组
        pdb_ids.append(pdbid)
        pdb_features.append(features)
    
    # 根据PDBid返回其对应的标签值（Ki, Kd, IC50）
    data_x = np.array(pdb_features, dtype=np.float32)
    data_y = np.array([pdbbind_dataset.sets[sets][_id] for _id in pdb_ids], dtype=np.float32)

    return data_x, data_y

In [29]:
def get_features():
    """ 返回数据库中的体素化结果x 并对应标签y值 """ 

    # 核心数据集选择
    core_ids = list(pdbbind_dataset.sets['core'].keys())
    # 精炼数据集选择
    #refined_ids = list(pdbbind_dataset.sets['refined'].keys()) 
    # 将精炼数据集和核心数据集独立
    #refined_ids = [i for i in refined_ids if i not in core_ids]
    
    # 执行批量体素化 
    print("Extracting features for the core set")
    core_x, core_y = get_pdb_features(core_ids, sets="core")
    #print("Extracting features for the refined set")
    #refined_x, refined_y = get_pdb_features(refined_ids)    
    
    return core_x, core_y

In [30]:
def main():
    # 核心数据集作为测试集
    test_x, test_y = get_features()
    # 划分训练集、验证集
    #train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=1)
    print("Shapes in the training, test and the validation set: ", test_x.shape)

    # 切片蛋白体素和小分子体素并分别保存
    #train_xp, train_xl = train_x[..., 0:8], train_x[..., 8:]
    #valid_xp, valid_xl = valid_x[..., 0:8], valid_x[..., 8:]
    test_xp, test_xl = test_x[..., 0:8], test_x[..., 8:]

    # 保存
    print("Saving the data in pretest.h5")
    h5f = h5py.File(os.path.join(data_dir, "test1.h5"), 'w')
    '''
    h5f.create_dataset('train_xp', data=train_xp)
    h5f.create_dataset('train_xl', data=train_xl)
    h5f.create_dataset('train_y', data=train_y)
    h5f.create_dataset('valid_xp', data=valid_xp)
    h5f.create_dataset('valid_xl', data=valid_xl)
    h5f.create_dataset('valid_y', data=valid_y)
    '''
    h5f.create_dataset('test_xp', data=test_xp)
    h5f.create_dataset('test_xl', data=test_xl)
    h5f.create_dataset('test_y', data=test_y)
    h5f.close()

In [31]:
if __name__=="__main__":main()

Extracting features for the core set


  0%|          | 0/290 [00:00<?, ?it/s]2023-08-31 14:21:39,997 - moleculekit.smallmol.smallmol - INFO - Adding any missing hydrogens (fixHs=True)
2023-08-31 14:21:40,025 - moleculekit.molecule - INFO - Removed 467 atoms. 2099 atoms remaining in the molecule.
2023-08-31 14:21:40,047 - moleculekit.molecule - INFO - Removed 4 atoms. 2095 atoms remaining in the molecule.
2023-08-31 14:21:43,361 - moleculekit.tools.preparation - WARNING - Dubious protonation state: the pKa of 2 residues is within 1.0 units of pH 7.4.
2023-08-31 14:21:43,362 - moleculekit.tools.preparation - WARNING - Dubious protonation state:    HIS    67 A (pKa= 6.90)
2023-08-31 14:21:43,364 - moleculekit.tools.preparation - WARNING - Dubious protonation state:    HIS    67 B (pKa= 6.75)
2023-08-31 14:21:43,934 - moleculekit.tools.atomtyper - INFO - Checking validity of Molecule before atomtyping. If it gives incorrect results or to improve performance disable it with validitychecks=False. Most of these checks can be pass

*** Open Babel Warning  in OpenBabel::OBMol::PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is C:\Users\Able9\AppData\Local\Temp\tmpimgv32g3.pdb)


  1%|▏         | 4/290 [00:49<1:17:20, 16.23s/it]2023-08-31 14:22:29,834 - moleculekit.smallmol.smallmol - INFO - Adding any missing hydrogens (fixHs=True)
2023-08-31 14:22:29,897 - moleculekit.molecule - INFO - Removed 893 atoms. 4205 atoms remaining in the molecule.
2023-08-31 14:22:36,791 - moleculekit.tools.preparation - WARNING - Dubious protonation state: the pKa of 7 residues is within 1.0 units of pH 7.4.
2023-08-31 14:22:36,791 - moleculekit.tools.preparation - WARNING - Dubious protonation state:    HIS     7 A (pKa= 6.46)
2023-08-31 14:22:36,791 - moleculekit.tools.preparation - WARNING - Dubious protonation state:    LYS   145 A (pKa= 8.24)
2023-08-31 14:22:36,791 - moleculekit.tools.preparation - WARNING - Dubious protonation state:    LYS   160 A (pKa= 8.22)
2023-08-31 14:22:36,791 - moleculekit.tools.preparation - WARNING - Dubious protonation state:    HIS   222 A (pKa= 7.24)
2023-08-31 14:22:36,791 - moleculekit.tools.preparation - WARNING - Dubious protonation state: 

*** Open Babel Warning  in OpenBabel::OBMol::PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is C:\Users\Able9\AppData\Local\Temp\tmpv0_hqrr9.pdb)


...

ERROR in  3utu   Found atoms with resnames ['VAL'] in the Molecule which can cause issues with the voxelization. Please make sure to only pass protein atoms and metals.


2023-08-31 15:16:31,150 - moleculekit.tools.preparation - WARNING - Dubious protonation state: the pKa of 2 residues is within 1.0 units of pH 7.4.
2023-08-31 15:16:31,151 - moleculekit.tools.preparation - WARNING - Dubious protonation state:    HIS   519 B (pKa= 6.92)
2023-08-31 15:16:31,152 - moleculekit.tools.preparation - WARNING - Dubious protonation state:    GLU   582 B (pKa= 7.37)
2023-08-31 15:16:31,626 - moleculekit.tools.atomtyper - INFO - Checking validity of Molecule before atomtyping. If it gives incorrect results or to improve performance disable it with validitychecks=False. Most of these checks can be passed by using the moleculekit.atomtyper.prepareProteinForAtomtyping function. But make sure you understand what you are doing.


*** Open Babel Warning  in OpenBabel::OBMol::PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is C:\Users\Able9\AppData\Local\Temp\tmp341bzgoc.pdb)


 98%|█████████▊| 284/290 [54:53<00:50,  8.46s/it]2023-08-31 15:16:33,758 - moleculekit.smallmol.smallmol - INFO - Adding any missing hydrogens (fixHs=True)
2023-08-31 15:16:33,787 - moleculekit.molecule - INFO - Removed 508 atoms. 2221 atoms remaining in the molecule.
2023-08-31 15:16:33,809 - moleculekit.molecule - INFO - Removed 1 atoms. 2220 atoms remaining in the molecule.
2023-08-31 15:16:37,461 - moleculekit.tools.preparation - WARNING - Dubious protonation state: the pKa of 2 residues is within 1.0 units of pH 7.4.
2023-08-31 15:16:37,462 - moleculekit.tools.preparation - WARNING - Dubious protonation state:    HIS    57 A (pKa= 7.24)
2023-08-31 15:16:37,463 - moleculekit.tools.preparation - WARNING - Dubious protonation state:    ASP    70 A (pKa= 7.61)
2023-08-31 15:16:37,906 - moleculekit.tools.atomtyper - INFO - Checking validity of Molecule before atomtyping. If it gives incorrect results or to improve performance disable it with validitychecks=False. Most of these checks c

Shapes in the training, test and the validation set:  (288, 24, 24, 24, 16)
Saving the data in pretest.h5
